# Estimations

## Prediction of MFPT with ISR

Here we provide an example of predicting the MFPT with ISR using available trajectories without resetting.
The given example is for free diffusion in one-dimension from $x=1$ to $x=0$, with a resetting rate $r=1$ at $x>0$ and $r=0$ otherwise.
For this example, there is an analytic solution in Ref. 28 of the manuscript. 

In [3]:
import numpy as np

N = 1000                   # Number of trajectories
dt = 0.001                  # Time step
D = 1                       # Diffusion coefficient
scale = np.sqrt(2 * dt * D) # Coefficient of normal noise in Langevin dynamics
r = 1                       # Resetting rate
x0 = 1                      # Initial position

PSI = 0 
t_f = 0
t_hat = 0

for i in range(N):

    # Sample a diffusive trajectory from x = 1 to the origin
    x = np.array([x0] + list(np.random.normal(scale = scale, size = 999999))).cumsum()
    ni = np.argmax(x < 0) if 0 != np.argmax(x < 0) else len(x)
    x = x[:ni]
    
    p = r * (x > 1) * dt                          # The probability of resetting given that no resetting occurred previously (Eq. 3)
    psi = (1 - p).cumprod() / (1 - p)             # Survival probability of trajectory i through Eq. 4
    psi = psi * (psi > 0) 
    
    PSI += psi[-1]                                # Adding the contribution of trajectory i in Eq. 5
    t_f += psi[-1] * ni * dt                      # Adding the contribution of trajectory i in Eq. 7
    t_hat += (p * psi * np.arange(ni)).sum() * dt # Adding the contribution of trajectory i in Eq. 9

# Normalization

PSI = PSI / N
M = 1 / PSI # Eq. 6
t_f = t_f / N / PSI
t_hat = t_hat / N / (1 - PSI)

MFPT = (M - 1) * t_hat + t_f                                       # The MFPT through Eq. 2
analytic = -x0 ** 2 / (2 * D) + x0 * (x0 / D + 1 / np.sqrt(r * D)) # The analytic MFPT from ref. 28
print("Analytic value: ", analytic, "; Prediction with 1000 samples: ", MFPT)

Analytic value:  1.5 ; Prediction with 1000 samples:  1.4720978134984999


## Kinetics inference

Next we provide an example of kinetics inference through Taylor expansion of the MFPT as a function of resetting rate.
First, we use the procedure above to predict the MFPT under resetting rates $> r^*$ from simulations with resetting rate $r^*$. Below we already provide these values. In the second step, we use them to estimate the derivatives at $r = r^*$ with finite difference method. Finally, we extrapolate the fitted function to $r = 0$ to estimate the unbiased MFPT.

In [6]:
def forwardExpansion(r,r0,delta,vals):
    """
    r = the rate at which to estimate the MFPT
    r0 = the rate used for the samples
    delta = the distance between grid points
    vals = the MFPT at the grid points
    """
    first = (-49/20*vals[0] + 6*vals[1] -15/2*vals[2] + 20/3*vals[3] -15/4*vals[4] + 6/5*vals[5] -1/6*vals[6])/delta
    second = (469/90*vals[0] -223/10*vals[1] +879/20*vals[2] -949/18*vals[3] +41*vals[4] -201/10*vals[5] +1019/180*vals[6]-7/10*vals[7])/(delta**2)
    third = (-801/80*vals[0] +349/6*vals[1] -18353/120*vals[2] +2391/10*vals[3] -1457/6*vals[4] +4891/30*vals[5] -561/8*vals[6]+527/30*vals[7]-469/240*vals[8])/(delta**3)
    fourth = (1069/80*vals[0] -1316/15*vals[1] +15289/60*vals[2] -2144/5*vals[3] +10993/24*vals[4] -4772/15*vals[5] +2803/20*vals[6]-536/15*vals[7]+967/240 *vals[8])/(delta**4)
    
    return vals[0] + first*(r-r0) + second*(r-r0)**2/2 + third*(r-r0)**3/6 + fourth*(r-r0)**4/24

predictions = [3.606,2.690,2.181,1.885,1.688,1.544,1.433,1.3452508,1.273]
rate = 1

print(f"The predicted MFPT is {forwardExpansion(0,rate,rate,predictions)} ns")

The predicted MFPT is 5.102528999166659 ns
